# Tracker Per-LS DC Notebook

This notebook is intended to provide tools that enable an effective per lumisection (LS) analysis of runs for data certification. It utilizes the DIALS Python API to fetch the monitoring element histograms, as well as the OMS API to obtain metadata on the run including the trigger rate. These tools offer the option to display the integral of a specified reference run in addition to the run under evaluation. This notebook is intended to be run in SWAN. However, in the case you wish to run it in a local virtual environment, a `pyproject.toml` is included which specifies all of the basic dependencies which you can install using Poetry.

Note that in order to run the OMS API you will need to have a json file with the client ID (`API_CLIENT_ID`) and secret (`API_CLIENT_SECRET`). For more information on how to obtain these, you can take a look at these [slides](https://indico.cern.ch/event/997758/contributions/4191705/attachments/2173881/3670409/OMS%20CERN%20OpenID%20migration%20-%20update.pdf).

## Setup

In [ ]:
# DIALS API
import cmsdials
from cmsdials.auth.client import AuthClient
from cmsdials.auth.bearer import Credentials
from cmsdials import Dials
from cmsdials.filters import LumisectionHistogram1DFilters, LumisectionHistogram2DFilters

auth = AuthClient()
token = auth.device_auth_flow()
creds = Credentials.from_authclient_token(token)

creds = Credentials.from_creds_file()
dials = Dials(creds)

In [ ]:
!git clone https://github.com/roy-cruz/OMSapi
!pip3 install ./OMSapi

In [ ]:
# OMS API
import json
import os

with open("../clientid.json", "r") as file:
    secrets = json.load(file)

os.environ["API_CLIENT_ID"] = secrets["API_CLIENT_ID"]
os.environ["API_CLIENT_SECRET"] = secrets["API_CLIENT_SECRET"]

import oms

oms_fetch = oms.oms_fetch()

In [ ]:
# Plotly
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import plot

In [ ]:
# Other useful libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import awkward as ak

# Misc.
from typing import List
import numpy.typing as npt
import ipywidgets as widgets
from IPython.display import display, clear_output

In [ ]:
# Importing a list of the available monitoring elements
from MEs import available_MEs
print("Available MEs: \n")
available_MEs

In [ ]:
MES_1D = [
    'PixelPhase1/Tracks/charge_PXBarrel',
    'PixelPhase1/Tracks/charge_PXForward',
    'PixelPhase1/Tracks/PXBarrel/charge_PXLayer_1',
    'PixelPhase1/Tracks/PXBarrel/charge_PXLayer_2',
    'PixelPhase1/Tracks/PXBarrel/charge_PXLayer_3',
    'PixelPhase1/Tracks/PXBarrel/charge_PXLayer_4',
    'PixelPhase1/Tracks/PXForward/charge_PXDisk_+1',
    'PixelPhase1/Tracks/PXForward/charge_PXDisk_+2',
    'PixelPhase1/Tracks/PXForward/charge_PXDisk_+3',
    'PixelPhase1/Tracks/PXForward/charge_PXDisk_-1',
    'PixelPhase1/Tracks/PXForward/charge_PXDisk_-2',
    'PixelPhase1/Tracks/PXForward/charge_PXDisk_-3',
    'PixelPhase1/Tracks/PXBarrel/size_PXLayer_1',
    'PixelPhase1/Tracks/PXBarrel/size_PXLayer_2',
    'PixelPhase1/Tracks/PXBarrel/size_PXLayer_3',
    'PixelPhase1/Tracks/PXBarrel/size_PXLayer_4',
    'PixelPhase1/Tracks/PXForward/size_PXDisk_+1',
    'PixelPhase1/Tracks/PXForward/size_PXDisk_+2',
    'PixelPhase1/Tracks/PXForward/size_PXDisk_+3',
    'PixelPhase1/Tracks/PXForward/size_PXDisk_-1',
    'PixelPhase1/Tracks/PXForward/size_PXDisk_-2',
    'PixelPhase1/Tracks/PXForward/size_PXDisk_-3',
    'SiStrip/MechanicalView/TID/PLUS/wheel_1/Summary_ClusterStoNCorr__OnTrack__TID__PLUS__wheel__1',
    'SiStrip/MechanicalView/TID/PLUS/wheel_2/Summary_ClusterStoNCorr__OnTrack__TID__PLUS__wheel__2',
    'SiStrip/MechanicalView/TID/PLUS/wheel_3/Summary_ClusterStoNCorr__OnTrack__TID__PLUS__wheel__3',
    'SiStrip/MechanicalView/TID/MINUS/wheel_1/Summary_ClusterStoNCorr__OnTrack__TID__MINUS__wheel__1',
    'SiStrip/MechanicalView/TID/MINUS/wheel_2/Summary_ClusterStoNCorr__OnTrack__TID__MINUS__wheel__2',
    'SiStrip/MechanicalView/TID/MINUS/wheel_3/Summary_ClusterStoNCorr__OnTrack__TID__MINUS__wheel__3',
]

MES_2D = [
    'PixelPhase1/Phase1_MechanicalView/PXBarrel/digi_occupancy_per_SignedModuleCoord_per_SignedLadderCoord_PXLayer_1',
    'PixelPhase1/Phase1_MechanicalView/PXBarrel/digi_occupancy_per_SignedModuleCoord_per_SignedLadderCoord_PXLayer_2',
    'PixelPhase1/Phase1_MechanicalView/PXBarrel/digi_occupancy_per_SignedModuleCoord_per_SignedLadderCoord_PXLayer_3',
    'PixelPhase1/Phase1_MechanicalView/PXBarrel/digi_occupancy_per_SignedModuleCoord_per_SignedLadderCoord_PXLayer_4',
    'PixelPhase1/Phase1_MechanicalView/PXForward/digi_occupancy_per_SignedDiskCoord_per_SignedBladePanelCoord_PXRing_1',
    'PixelPhase1/Phase1_MechanicalView/PXForward/digi_occupancy_per_SignedDiskCoord_per_SignedBladePanelCoord_PXRing_2',
    'PixelPhase1/Tracks/clusterposition_zphi_ontrack',
    'PixelPhase1/Tracks/PXBarrel/clusterposition_zphi_ontrack_PXLayer_1',
    'PixelPhase1/Tracks/PXBarrel/clusterposition_zphi_ontrack_PXLayer_2',
    'PixelPhase1/Tracks/PXBarrel/clusterposition_zphi_ontrack_PXLayer_3',
    'PixelPhase1/Tracks/PXBarrel/clusterposition_zphi_ontrack_PXLayer_4',
    'PixelPhase1/Tracks/PXForward/clusterposition_xy_ontrack_PXDisk_+1',
    'PixelPhase1/Tracks/PXForward/clusterposition_xy_ontrack_PXDisk_+2',
    'PixelPhase1/Tracks/PXForward/clusterposition_xy_ontrack_PXDisk_+3',
    'PixelPhase1/Tracks/PXForward/clusterposition_xy_ontrack_PXDisk_-1',
    'PixelPhase1/Tracks/PXForward/clusterposition_xy_ontrack_PXDisk_-2',
    'PixelPhase1/Tracks/PXForward/clusterposition_xy_ontrack_PXDisk_-3',
]

## Helper Functions
Put here any functions you think would help you in your DC. We have put here some useful functions for plotting and processing of the data.

In [ ]:
def extractdata(queryrslt: cmsdials.clients.h2d.models.PaginatedLumisectionHistogram2DList, rtn_df=False) -> np.ndarray:
    """
    Transforms data fetched from dials into a numpy array and a dataframe (second one optional).
    """
    data_df = pd.DataFrame(queryrslt.dict()["results"])
    data_df.sort_values(["run_number", "ls_number"], inplace=True)

    data_dict = {}
    
    for run in data_df["run_number"].unique():
        data_dict[run] = {}
        data_dict[run]
        for me in data_df[data_df["run_number"]==run]["me"].unique():
            me_arr = data_df[data_df["me"] == me]["data"].to_numpy(dtype=np.ndarray)
            me_arr = np.array([np.array(x) for x in me_arr])
            data_dict[run][me] = me_arr
    if rtn_df:
        return (data_dict, data_df)
    else:
        return data_dict

In [ ]:
def normalize_by_trig(data_dict, trigger_rates: np.ndarray) -> np.ndarray:
    normalized_data = {}
    mes = list(data_dict.keys())
    for me in mes:
        if me in MES_1D:
            normalized_data[me] = data_dict[me] / trigger_rates[:, np.newaxis]
        elif me in MES_2D:
            n = data_dict[me].shape[1]
            m = data_dict[me].shape[2]
            normalized_data[me] = data_dict[me] / np.repeat(trigger_rates[:, np.newaxis], n * m, axis=1).reshape(-1, n, m)
    return normalized_data

In [ ]:
def plotinteractive1D(
        me_dict, 
        bin_locs: List[np.ndarray], 
        x_labels: List[str], 
        y_labels: List[str],
        fig_title: str,
        trigger_rates = None,
        normalized_area = True, 
        normalized_trig = True,
        using_ref = True,
        ref = None,
        show = True,
        ):
    """
    For plotting multiple 1D histograms
    """

    if using_ref:
        if ref is None:
            raise ValueError("It was indicated that a reference would be used, but none was provided.")
        if not normalized_area:
            raise ValueError("Reference will be normalized, but data won't be.")
    
    mes = list(me_dict.keys())
    num_mes = len(mes)
    
    if not (num_mes == len(bin_locs) == len(x_labels) == len(y_labels)):
        raise ValueError("All input lists must have the same length.")

    if normalized_trig:
        if trigger_rates is None:
            raise ValueError("No trigger rate array given.")
        me_dict = normalize_by_trig(me_dict, trigger_rates)
    
    if normalized_area:
        for me in mes:
            me_dict[me] = me_dict[me] / me_dict[me].sum(axis=1, keepdims=True)

    num_rows = (num_mes + 1) // 2
    num_cols = 2 if num_mes > 1 else 1
    num_lss = len(me_dict[mes[0]])

    # Fig object creation
    fig = make_subplots(
        rows=num_rows, cols=num_cols, 
        subplot_titles=mes,
        vertical_spacing=0.15,
        horizontal_spacing=0.05
    )

    # Add first LS
    ref_traces = []
    for i, me in enumerate(mes):
        row = (i // num_cols) + 1
        col = (i % num_cols) + 1
        fig.add_trace(go.Bar(x=bin_locs[i], y=me_dict[me][0], name=me[i]), row=row, col=col)
        fig.update_xaxes(title_text=x_labels[i], row=row, col=col)
        fig.update_yaxes(title_text=y_labels[i], row=row, col=col)
        if using_ref: 
            ref_traces.append(go.Scatter(x=bin_locs[i], y=ref[me], name="Reference", mode="lines"))
    if using_ref:
        for i, ref_trace in enumerate(ref_traces):
            row = (i // num_cols) + 1
            col = (i % num_cols) + 1
            fig.add_trace(ref_trace, row=row, col=col)

    # Make steps to update to the rest of the LSs
    steps = []
    for i in range(num_lss):
        step = {
            "method": "restyle",
            "args": [
                {"y": [me_dict[me][i,:] for me in mes]},
                np.arange(num_mes)  # Indices of the traces to modify
            ],
            "label": f"LS {i+1}"
        }
        steps.append(step)
        
    sliders = [{
        "active": 0,
        "currentvalue": {"prefix": 'LS: '},
        "pad": {"t": 50},
        "steps": steps
    }]

    # Add elements and update layout of fig
    fig.update_layout(
        sliders=sliders,
        title_text=fig_title,
        bargap=0,
        showlegend=False,
        width=1600,
        height=900
    )

    for i, me in enumerate(mes):
        row = (i // num_cols) + 1
        col = (i % num_cols) + 1
        max_y = me_dict[me].max()
        fig.update_yaxes(range=[0, max_y], row=row, col=col)

    if show:
        fig.show()
    else:
        return fig

In [ ]:
def plotheatmaps1D(
    me_dict, 
    bin_locs: List[np.ndarray], 
    x_labels: List[str], 
    y_labels: List[str],
    fig_title: str,
    trigger_rates = None,
    normalized_trig = False,
    show = True,
    ):

    mes = list(me_dict.keys())
    num_mes = len(mes)

    if not (len(bin_locs) == len(x_labels) == len(y_labels) == num_mes):
        raise ValueError("All input lists must have the same length.")

    # Normalizing by trigger rate
    if normalized_trig:
        if trigger_rates is None:
            raise ValueError("No trigger rate array given.")
        me_dict = normalize_by_trig(me_dict, trigger_rates)

    num_rows = (num_mes + 1) // 2
    num_cols = 2 if num_mes > 1 else 1
    num_lss = len(me_dict[mes[0]])

    # Making figure object
    fig = make_subplots(
        rows=num_rows, cols=num_cols, 
        subplot_titles = mes,
        vertical_spacing = 0.1,
        horizontal_spacing = 0.1
    )

    # Adding heatmap trace to figure
    for i, me in enumerate(mes):
        row = (i // 2) + 1
        col = (i % 2) + 1
        fig.add_trace(
            go.Heatmap(z=me_dict[me], x=bin_locs[i], colorscale="Viridis", showscale=False),
            row=row, col=col,
        )
        fig.update_xaxes(title_text=x_labels[i], row=row, col=col)
        fig.update_yaxes(title_text=y_labels[i], row=row, col=col)

    # Adding layour elements to figure
    fig.update_layout(
        title_text=fig_title,
        title_font={"size": 24},
        height=1100,
        width=1100,
        annotations = [dict(text = me, font={"size": 14}, showarrow=False) for me in mes],
    )

    fig.update_yaxes(autorange="reversed")

    if show:
        fig.show()
    else:
        return fig

In [ ]:
def multiplotinteractive2D(
    me_dict,
    x_bin_locs: List[np.array], 
    y_bin_locs: List[np.array],
    x_labels: List[str], 
    y_labels: List[str],
    fig_title: str,
    trigger_rates = None,
    normalized_trig = False,
    show = True,
    ):
    """
    Interactive occupancy maps
    """
    
    mes = list(me_dict.keys())
    num_mes = len(mes)
    num_lss = len(me_dict[mes[0]])

    num_rows = (num_mes + 1) // 2
    num_cols = 2 if num_mes > 1 else 1

    if normalized_trig:
        if trigger_rates is None:
            raise ValueError("No trigger rate array given.")
        me_dict = normalize_by_trig(me_dict, trigger_rates)

    fig = make_subplots(
        rows=num_rows, cols=num_cols, 
        subplot_titles=mes, 
        vertical_spacing = 0.1
    )
    
    for i, me in enumerate(mes):
        row = (i // 2) + 1
        col = (i % 2) + 1
        fig.add_trace(
            go.Heatmap(
                z = me_dict[me], 
                x = x_bin_locs[i],
                y = y_bin_locs[i],
                colorscale = "Viridis",
                zmin = me_dict[me].min() if isinstance(me_dict[me], np.ndarray) else ak.min(me_dict[me]),
                zmax = me_dict[me].max() if isinstance(me_dict[me], np.ndarray) else ak.max(me_dict[me]),
                showscale=False
            ),
            row = i // 2 + 1, col = i % 2 + 1
        )
        fig.update_xaxes(title_text=x_labels[i], row=row, col=col)
        fig.update_yaxes(title_text=y_labels[i], row=row, col=col)


    # Making the steps to update to the rest of the LSs
    steps = []
    for i in range(num_lss):
        step = dict(
            method = "restyle",
            args = [
                dict(
                    z = [me_dict[me][i] for me in mes],
                ),
                np.arange(num_mes)  # Indices of the traces to modify
            ],
            label = str(i+1)
        )
        steps.append(step)
        
    sliders = [
        dict(
            active = 0,
            currentvalue = dict(prefix = "LS: "),
            pad = dict(t = 50),
            steps = steps
        )
    ]

    # Button
    play_button = dict(
        type = "buttons",
        showactive = False,
        buttons = [
            dict(
                label = "Play",
                method = "animate",
                args = [
                    None, 
                    dict(
                        frame = dict(duration = 500, redraw = True),  # Speed of playback in milliseconds per step
                        fromcurrent = True,
                        transition = dict(duration = 300),
                        mode = "immediate"
                    ),
                ]
            )
        ]
    )

    # Define frames for animation 
    frames = []
    for i in range(num_lss):
        frame = dict(
            data=[
                go.Heatmap(z = me_dict[me][i]) for me in mes
            ],
            name = str(i+1),
            traces = np.arange(num_mes),
            layout = dict(sliders=[dict(active = i)])  # Update the slider position
        )
        frames.append(frame)

    fig.frames = frames

    fig.update_annotations(font = dict(size = 12))
    fig.update_layout(
        updatemenus = [play_button],
        sliders = sliders,
        title_text = fig_title,
        bargap = 0,
        showlegend = False,
        width = 1200,
        height = 1000,
    )

    if show:
        fig.show()
    else:
        return fig


In [ ]:
def integratenormalize (data_dict):
    mes = list(data_dict.keys())
    normalized = {}

    for me in mes:
        normalized[me] = data_dict[me].sum(axis=0)
        normalized[me] = normalized[me] / normalized[me].sum(axis=0, keepdims=True)

    return normalized

In [ ]:
def plotintegrated1D(
    me_dict,
    bin_locs,
    x_labels,
    y_labels,
    fig_title,
    ls_filter = None,
    trigger_rates = None,
    normalized_trig = True,
    using_ref = True,
    ref = None,
    show = True
):
    if using_ref:
        if ref is None:
            raise ValueError("It was indicated that a reference would be used, but none was provided.")
    
    mes = list(me_dict.keys())
    num_mes = len(mes)

    if not (num_mes == len(bin_locs) == len(x_labels) == len(y_labels)):
        raise ValueError("All input lists must have the same length.")

    if normalized_trig:
        if trigger_rates is None:
            raise ValueError("No trigger rate array given.")
        me_dict = normalize_by_trig(me_dict, trigger_rates)
    
    if isinstance(ls_filter, list):
        for me in mes:
            me_dict[me] = me_dict[me][ls_filter]
    
    me_dict = integratenormalize(me_dict)

    num_rows = (num_mes + 1) // 2
    num_cols = 2 if num_mes > 1 else 1
    num_lss = len(me_dict[mes[0]])

    # Fig object creation
    fig = make_subplots(
        rows=num_rows, cols=num_cols, 
        subplot_titles=mes,
        vertical_spacing=0.15,
        horizontal_spacing=0.05
    )
    
    # Add first LS
    ref_traces = []
    for i, me in enumerate(mes):
        row = (i // num_cols) + 1
        col = (i % num_cols) + 1
        fig.add_trace(go.Bar(x=bin_locs[i], y=me_dict[me], name=mes[i]), row=row, col=col)
        fig.update_xaxes(title_text=x_labels[i], row=row, col=col)
        fig.update_yaxes(title_text=y_labels[i], row=row, col=col)
        if using_ref: 
            ref_traces.append(go.Scatter(x=bin_locs[i], y=ref[me], name="Reference", mode="lines"))
    for i, ref_trace in enumerate(ref_traces):
        row = (i // num_cols) + 1
        col = (i % num_cols) + 1
        fig.add_trace(ref_trace, row=row, col=col)

    # Add elements and update layout of fig
    fig.update_layout(
        title_text=fig_title,
        bargap=0,
        showlegend=True,
        width=1600,
        height=900
    )

    for i, me in enumerate(mes):
        row = (i // num_cols) + 1
        col = (i % num_cols) + 1
        max_y = me_dict[me].max()
        fig.update_yaxes(range=[0, max_y], row=row, col=col)

    if show:
        fig.show()
    else:
        return fig

## Using OMS to Obtain Metadata

Using the OMS API, we can access important information regarding the run conditions and other information about the run. The available endpoints are:

* `lumisections`
* `runs`
* `fills`
* `datasetrates`

If you wish to access per-LS trigger rate, you can use the last one as shown below.

In [ ]:
runnb = 380238

oms_json = oms_fetch.get_oms_json(api_endpoint="lumisections", runnb=runnb)
oms_df = oms.oms_utils.makeDF(oms_json)
oms_df.columns

In [ ]:
oms_df.head(5)

In [ ]:
# Getting per-LS trigger rates

extrafilter = dict(
    attribute_name="dataset_name",
    value="ZeroBias",
    operator="EQ",
)

attributes = [
    'start_time', 
    'last_lumisection_number', 
    'rate', 
    'run_number',
    'last_lumisection_in_run', 
    'first_lumisection_number', 
    'dataset_name',
    'cms_active', 
    'events'
]

omstrig_json = oms.get_oms_data(
    oms_fetch.omsapi, 
    'datasetrates', 
    runnb,
    extrafilters=[extrafilter]
)

omstrig_df = oms.oms_utils.makeDF(omstrig_json)

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x = omstrig_df["last_lumisection_number"],
        y = omstrig_df["rate"],
    )
)

fig.update_layout(
    title = f"Trigger rate per-LS for Run {runnb}",
    xaxis_title = "Lumisection",
    yaxis_title = "Trigger rate",
)

fig.show()

We store the trigger rate in an array to use later on.

In [ ]:
# Getting trigger rate array
trig_rate = np.array(omstrig_df["rate"])

## 1D Monitoring Elements

In [ ]:
LumisectionHistogram1DFilters?

We fetch data from DIALS as shown here. If you are unfamiliar with regex syntax, you can take a look at the following [cheat sheet](https://www.rexegg.com/regex-quickstart.html) for a quick overview.

In [ ]:
runnb = 380238
me__regex =  "PixelPhase1/Tracks/PXBarrel/charge_PXLayer_."
# me__regex = "SiStrip/MechanicalView/TID/PLUS/wheel_./Summary_ClusterStoNCorr__OnTrack__TID__PLUS__wheel_."

data1D = dials.h1d.list_all(
    LumisectionHistogram1DFilters(
        next_token = None,
        dataset_id = None,
        file_id = None,
        run_number = runnb,
        run_number__lte = None,
        run_number__gte = None,
        ls_number = None,
        ls_numbet__lte = None,
        ls_number__gte = None,
        me_id = None,
        entries__gte = None,
        dataset = "/ZeroBias/Run2024C-PromptReco-v1/DQMIO",
        dataset__regex = None,
        logical_file_name = None,
        logical_file_name__regex = None,
        me = None,
        me__regex = me__regex
    ),
    # max_pages=200
)

data1D_dict, data1D_df = extractdata(queryrslt=data1D, rtn_df=True)
MEs_fetched = list(data1D_df["me"].unique())

In [ ]:
# Hitogram bin locations are computed
bin_locs = []
for ME in MEs_fetched:
    x_min = data1D_df[data1D_df["me"] == ME]["x_min"].iloc[0]
    x_max = data1D_df[data1D_df["me"] == ME]["x_max"].iloc[0]
    x_bin = data1D_df[data1D_df["me"] == ME]["x_bin"].iloc[0]
    bin_locs.append(np.linspace(x_min, x_max, int(x_bin)))

Before we begin plotting, we could also get a reference run to have something to compare the distributions we get to. To be able to properly compare the reference with the run under consideration, we integrate it over the LSs and then normalize it such that the area under the curve is one.

In [ ]:
# Getting reference data
refrun = 379765

refdata1D = dials.h1d.list_all(
    LumisectionHistogram1DFilters(
        next_token = None,
        dataset_id = None,
        file_id = None,
        run_number = refrun,
        run_number__lte = None,
        run_number__gte = None,
        ls_number = None,
        ls_numbet__lte = None,
        ls_number__gte = None,
        me_id = None,
        entries__gte = None,
        dataset = "/ZeroBias/Run2024C-PromptReco-v1/DQMIO",
        dataset__regex = None,
        logical_file_name = None,
        logical_file_name__regex = None,
        me = None,
        me__regex = me__regex
    ),
    # max_pages=200
)

refdata1D_dict, refdata1D_df = extractdata(queryrslt=refdata1D, rtn_df=True)
refMEs_fetched = list(refdata1D_df["me"].unique())
refdata1D_integrated = integratenormalize(refdata1D_dict[379765])

`plotinteractive1D` offers the ability to immediately show the plot inside the notebook. However, unless you only intend to make one or two plots, this is not recommended as Plotly plots are resource intensive. It is recommended that you instead do as the following example where we set `show = False` so that the function returns the figure and that you then export the plot as an `html` file which you can then see in your browser.

In [ ]:
plotinteractive1D?

In [ ]:
x_labels = ["ClusterStoN"] * len(MEs_fetched)
y_labels = ["Count"] * len(MEs_fetched)
fig_title = f"Pixel Barrel Charge (Run {runnb})"
# fig_title = f"ClusterStoN (Run {runnb})"

In [ ]:
fig = plotinteractive1D(
    me_dict = data1D_dict[runnb], 
    bin_locs = bin_locs,
    x_labels = x_labels, 
    y_labels = y_labels,
    trigger_rates = trig_rate,
    fig_title = fig_title,
    using_ref = True,
    ref = refdata1D_integrated,
    normalized_area = True,
    normalized_trig = True,
    show = False
)

# Export plot to html
plot(fig, filename="me_plot.html")

### Heatmaps

By "stacking" 1D histograms, we can create heatmaps which give us an idea of how the run evolved through time as data was being taken.

In [ ]:
plotheatmaps1D?

In [ ]:
hm_xlabels = ["Charge (e)"] * len(MEs_fetched)
hm_ylabels = ["Lumisection"] * len(MEs_fetched)
hm_fig_title = f"Pixel Barrel Charge (Run {runnb})"

fig = plotheatmaps1D(
    me_dict = data1D_dict[runnb],
    bin_locs = bin_locs,
    x_labels = hm_xlabels,
    y_labels = hm_ylabels,
    fig_title = hm_fig_title,
    trigger_rates = trig_rate,
    normalized_trig = True,
    show=False
)

plot(fig, filename="me_heat_plot.html")

## 2D Monitoring Elements

In [ ]:
LumisectionHistogram2DFilters?

2D monitoring elements are also available in DIALS and they can be accessed as shown below.

In [ ]:
runnb = 380238

data2D = dials.h2d.list_all(
    LumisectionHistogram1DFilters(
        next_token = None,
        dataset_id = None,
        file_id = None,
        run_number = runnb,
        run_number__lte = None,
        run_number__gte = None,
        ls_number = None,
        ls_numbet__lte = None,
        ls_number__gte = None,
        me_id = None,
        entries__gte = None,
        dataset = "/ZeroBias/Run2024C-PromptReco-v1/DQMIO",
        dataset__regex = None,
        logical_file_name = None,
        logical_file_name__regex = None,
        me = None,
        me__regex = 'PixelPhase1/Phase1_MechanicalView/PXBarrel/digi_occupancy_per_SignedModuleCoord_per_SignedLadderCoord_PXLayer_.'
    ),
    # max_pages=200
)

data2D_dict, data2D_df = extractdata(queryrslt=data2D, rtn_df=True)
MEs2D_fetched = list(data2D_df["me"].unique())

In [ ]:
# Computing histogram bin locations in x and y
x_bin_locs = []
y_bin_locs = []
for ME in MEs2D_fetched:
    x_min = data2D_df[data2D_df["me"] == ME]["x_min"].iloc[0]
    x_max = data2D_df[data2D_df["me"] == ME]["x_max"].iloc[0]
    x_bin = data2D_df[data2D_df["me"] == ME]["x_bin"].iloc[0]
    x_bin_locs.append(np.linspace(x_min, x_max, int(x_bin)))

    y_min = data2D_df[data2D_df["me"] == ME]["y_min"].iloc[0]
    y_max = data2D_df[data2D_df["me"] == ME]["y_max"].iloc[0]
    y_bin = data2D_df[data2D_df["me"] == ME]["y_bin"].iloc[0]
    y_bin_locs.append(np.linspace(y_min, y_max, int(y_bin)))

Like `multiplotinteractive1D`, `multiplotinteractive2D` also provides an option to return the figure instead of immediately plotting. For 2D histograms, this is highly recommended even if you are thinking of only making a single plot, as 2D histograms are particularly resource intensive and can cause issues with the notebook.

In [ ]:
multiplotinteractive2D?

In [ ]:
# x_labels = ["SignedModuleCoord", "z"]
# y_labels = ["SignedLadderCoord", "phi"]
# fig_title = f"OnTrack Cluster Position & Digi Occupancy Map for PX Barrel (Run {runnb})"

x_labels = ["SignedModuleCoord"] * len(MEs2D_fetched)
y_labels = ["SignedLadderCoord"] * len(MEs2D_fetched)
fig_title = f"Digi Occupancy Map for PX Barrel (Run {runnb})"

fig2D = multiplotinteractive2D(
    me_dict = data2D_dict[380238], 
    x_bin_locs = x_bin_locs, 
    y_bin_locs = y_bin_locs,
    x_labels = x_labels, 
    y_labels = y_labels,
    fig_title = fig_title,
    trigger_rates = trig_rate,
    normalized_trig = True,
    show = False,
)

plot(fig2D, filename="me2D_plot.html")

## Integrated MEs

Suppose you found that the first 10 lumisections of a run have some sort of issue and you want to get the same plot the GUI would give you, but without including those LSs. This next section offers some tools to do just that. Run the following block of code to generate a slider widget which you can use to interactively select the LSs you wish to integrate over.

In [ ]:
range_slider = widgets.IntRangeSlider(
    value=[1, len(trig_rate)],
    min=1,
    max=len(trig_rate),
    step=1,
    description='Lumisections:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
)

output = widgets.Output()
display(output)

# List to store selected ranges
# Temporary list to hold currently selected, but unaded ranged
selected_range = []
# List of added ranges
final_ranges = []
# Full list of lss generated from list of added ranges
selected_lss = []

def ranges_overlap(range1, range2):
    start1, end1 = range1
    start2, end2 = range2
    return start1 <= end2 and start2 <= end1

# Slider interactivity functions
def on_slider_change(change):
    selected_range[:] = tuple(change['new'])
    update_output()

def update_output():
    with output:
        clear_output()
        print(f"Selected ranges: {selected_range}")

# Attch change handler to slider
range_slider.observe(on_slider_change, names='value')

display(range_slider, output)

# Buttons def
def clear_ranges(_):
    final_ranges.clear()
    update_output()

def apply_ranges(_):
    for frange in final_ranges:
        if ranges_overlap(frange, tuple(selected_range)):
            print("Error: Overlap of ranges. Try again.")
            break
    else:
        final_ranges.append(tuple(selected_range[:]))
        print(f"Range selected: {final_ranges}")
        
def get_lss(_):
    selected_lss.clear()
    for lsrange in final_ranges:
        selected_lss.extend(range(lsrange[0], lsrange[1]+1))
        selected_lss.sort()
    with output:
        print(f"Lumisection selection: {selected_lss}")

clear_button = widgets.Button(description="Clear Ranges")
clear_button.on_click(clear_ranges)

apply_button = widgets.Button(description="Apply Ranges")
apply_button.on_click(apply_ranges)

get_lss_button = widgets.Button(description="Get LSs")
get_lss_button.on_click(get_lss)

display(clear_button, apply_button, get_lss_button)

In [ ]:
x_labels = ["Charge (e)"] * len(data1D_dict[runnb].keys())
y_labels = ["Count"] * len(data1D_dict[runnb].keys())
fig_title = f"Pixel Barrel Charge (Run {runnb})"

In [ ]:
figintegrated = plotintegrated1D(
    me_dict = data1D_dict[runnb], 
    bin_locs = bin_locs,
    x_labels = x_labels, 
    y_labels = y_labels,
    fig_title = fig_title,
    ls_filter = selected_lss,
    trigger_rates = trig_rate,
    using_ref = True,
    ref = refdata1D_integrated,
    normalized_trig = True,
    show = False
)

plot(figintegrated, filename="integrated1D_plot.html")